In [ ]:
import requests

def Next_page(response):
    header=response.headers.get('Link', '')
    links = header.split(',')
    for link in links:
        if 'rel="next"' in link:
            next_page = link.split(';')[0].strip('<>')
            return next_page
    return None

def total(list):
    sum=0
    for url in list:
        next_page = url+'?per_page=100'
        while next_page != None:
            response = requests.get(next_page)
            sum = sum + len(response.json())
            next_page = Next_page(response)
    return sum

url = f"https://api.github.com/users/Kaggle/repos"
response = requests.get(url)
if response.status_code == 200:
    repos = response.json()
    
commits = []
stars = []
contributors = []
branches = []
tags = []
forks = []
releases = []
closed_issues = []
language_lines = {}

for repo in repos:
    repo_url = repo['url'] #find all urls in one repo
    response = requests.get(repo_url)
    if response.status_code == 200:
        repo_details = response.json()
    if repo_details: #get information about the repo
        commits.append(repo_details['commits_url'].split('{')[0]) #the commit url for each repository
        stars.append(repo_details['stargazers_count'])
        contributors.append(repo_details['contributors_url'])
        branches.append(repo_details['branches_url'].split('{')[0])
        tags.append(repo_details['tags_url'].split('{')[0])
        forks.append(repo_details['forks_count'])
        releases.append(repo_details['releases_url'].split('{')[0])
        closed_issues.append(repo_details['issues_url'].replace('{/number}', '?state=closed').split('{')[0])

        language_stats_url = repo_details['languages_url']
        response = requests.get(language_stats_url)
        if response.status_code == 200:
            language_data = response.json()
            for language, lines in language_data.items():
                language_lines[language] = language_lines.get(language, 0) + lines
                
total_commits = total(commits)
total_stars = sum(stars)
total_contributors = total(contributors)
total_branches = total(branches)
total_tags = total(tags)
total_forks = sum(forks)
total_releases = total(releases)
total_closed_issues = total(closed_issues)
number = len(commits)

print(f"Total Commits: {total_commits}, Median Commits: {total_commits/number:.2f}")
print(f"Total Stars: {total_stars}, Median Stars: {total_stars/number:.2f}")
print(f"Total Contributors: {total_contributors}, Median Contributors: {total_contributors/number:.2f}")
print(f"Total Branches: {total_branches}, Median Branches: {total_branches/number:.2f}")
print(f"Total Tags: {total_tags}, Median Tags: {total_tags/number:.2f}")
print(f"Total Forks: {total_forks}, Median Forks: {total_forks/number:.2f}")
print(f"Total Releases: {total_releases}, Median Releases: {total_releases/number:.2f}")
print(f"Total Closed Issues: {total_closed_issues}, Median Closed Issues: {total_closed_issues/number:.2f}")
print(f"Total Environments: {len(language_lines.items())}, Median Environments: {len(language_lines.items())/number:.2f}")
print("\nLanguage-wise lines of code:")
sum=0
for language, lines in language_lines.items():
    sum = sum + lines
    print(f"{language}: {lines}")
print("\nmedian number of lines:")
print(f"{sum/number:.2f}")